# Warping tutorial
## f_playing_with_phase_comp

##### May 2020
###### Eva Chamorro - Daniel Zitterbart - Julien Bonnel

## 1. Import packages

In [1]:
import os
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
%matplotlib widget
from matplotlib import interactive
from matplotlib.path import Path
from scipy import interpolate
from scipy.fftpack import fft, ifft
from scipy.signal import hilbert
from ipywidgets import interact, interact_manual
from warping_functions import *
from time_frequency_analysis_functions import *
from bbox_select import *
from pts_select import *
import warnings
warnings.filterwarnings('ignore')

## 2. Load simulated signal

In [2]:
data = sio.loadmat(os.getcwd()+ '/sig_pek_for_warp.mat')


'''
    s_t: propagated modes in a Pekeris waveguide with parameters
    c1, c2, rho1, rho2: sound speed / density
        D: depth
        r: range
        zs, zr: source/receiver depth
    s_t_dec: same than s_t, except that time origin has been set for warping
    fs: sampling frequency
    
   
     NB: one can run optional_create_simulated_signal.m to generate another
     simulated signal
'''

# Select variables
s_t=data['s_t']
fs=data['fs']
s_t_dec=data['s_t_dec']
r=data['r']
c1=data['c1']

## 3. Process signal 

In [3]:
# The first sample of s_t_dec corresponds to time r/c1
# Make the signal shorter, no need to keep samples with zeros
N_ok=250
ir_ok=s_t_dec[:,0:N_ok] ### this is the impulse response of the waveguide
print('Continue')

Continue


## 4. Source signal creation

In [4]:
## Source signal creation
X,IFLAW_source=fmpar(200,np.array([1,0.5]),np.array([100,0.15]),np.array([200,0.01]))


# source signal is a parabolic modulated FM signal
source=X*(IFLAW_source+1)
N_source=len(source)

print('Continue')

Continue


## 5. Propagated signal

In [5]:
# Propagated signal = time-convolution between source and impulse response = multiplication in the frequency domain
source_f=fft(source, N_ok)
source_f=source_f[:,np.newaxis]
ir_f=fft(ir_ok,N_ok)
s_f=source_f*np.transpose(ir_f)
s_ok=ifft(s_f,N_ok, axis=0) #propagated signal

# STFT computation
NFFT=1024
N_window=31 # you need a short window to see the modes
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
d=np.hamming(N_window)
d=d[:,np.newaxis]
tfr=tfrstft(s_ok,b,NFFT,d)
source_1=source[:,np.newaxis]
tfr_source=tfrstft(source_1,b,NFFT,d)

spectro=abs(tfr)**2
spectro_source=abs(tfr_source)**2


# Time and frequency axis of the original signal
time=np.arange(0,N_ok)/fs
freq=np.arange(0,NFFT)*fs/NFFT



time_init=time
freq_init=freq
spectro_init=spectro

## 6.1 Phase compensation

In [6]:
print('This is the same example as has been shown previously (see e_warping_and_filtering_and_phase_comp.m)')
print('This time, we will approximate the source TF law with 3 linear pieces.')
print('As explained in Sec V C, an easy way to do so is to roughly follow the TF contour of mode 1')
print('Let us try')
print('')
print('')

spectro_s=spectro[:512,:]
freq_s=freq[0,:512]
time_s=time[0,:]

print('Click 4 times on the spectrogram to define your 3 linear pieces')
print('In this case, the easiest is probably to start at early-times/high frequencies,')
print('and to progress toward increasing times and decreasing frequencies')
print('(roughly follow the black line from the previous example)')

interactive(True)
point= pts_select(spectro_s,time_s,freq_s)

print('')
print('When you have click 4 times on the spectrogram, click "disconnect mpl" and continue') 

This is the same example as has been shown previously (see e_warping_and_filtering_and_phase_comp.m)
This time, we will approximate the source TF law with 3 linear pieces.
As explained in Sec V C, an easy way to do so is to roughly follow the TF contour of mode 1
Let us try


Click 4 times on the spectrogram to define your 3 linear pieces
In this case, the easiest is probably to start at early-times/high frequencies,
and to progress toward increasing times and decreasing frequencies
(roughly follow the black line from the previous example)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())


When you have click 4 times on the spectrogram, click "disconnect mpl" and continue


In [7]:
# we extract the points selected 
pts=point.selected_points
c=np.array(pts)
ttt=c[:,0]
fff=c[:,1]

print('The black line is your best guess of the source signal')
print('Now, let us do phase compensation to transform our received signal')
print('into something that looks like the impulse response of the waveguide')

### let's plot it on top of the received signal
plt.figure(figsize=(7,5))
plt.imshow(spectro, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto', origin='low')
plt.ylim([0, fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Received signal')
plt.plot(ttt,fff,'black', linewidth=3)
plt.show()

### convert to samples and reduced frequencies
ttt_s=np.round(ttt*fs)+1
fff_s=fff/fs
n_click=4

### create the piecewise linear-FM source
for ii in range (1,n_click):
    ifl=np.linspace(fff_s[0,ii-1],fff_s[0,ii],int(ttt_s[0,ii]-ttt_s[0,ii-1]))
    
    if ii==1:
        iflaw=ifl
    else:
        iflaw=np.concatenate((iflaw , ifl[1:]))

iflaw=iflaw[:, np.newaxis]

source_est, IFLAW_est=fmodany(iflaw) 
source_est=source_est[:,np.newaxis]
source_est_f=fft(source_est,N_ok,axis=0) ### estimated source signal in the frequency domain
phi=np.angle(source_est_f)   ### phase of the estimated source signal in the frequency domain

print('Continue')

The black line is your best guess of the source signal
Now, let us do phase compensation to transform our received signal
into something that looks like the impulse response of the waveguide


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue


## 6.2 Phase correction

In [8]:
## Phase correction
sig_prop_f=fft(s_ok,axis=0)
i=complex(0,1)
sig_rec_f=sig_prop_f*np.exp(-i*phi)  # note that only source phase is deconvoluted
sig_rec_t=ifft(sig_rec_f,axis=0)  #Signal in time domain after source deconvolution


# Figure
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
d=np.hamming(N_window)
d=d[:,np.newaxis]

tfr_sig_rec=tfrstft(sig_rec_t,b,NFFT,d)


print('Here is the result')
print('If you think it looks like an impulse response, continue to the next cell')
print('If you want to redo the phase compensation,go back to the cell "6.1 Phase compensation" ')
print('(you will have the opportunity to modify the time origin later ')


plt.figure(figsize=(7,5))
plt.imshow(abs(tfr_sig_rec)**2,extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto', origin='low')
plt.ylim([0,fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Signal after phase compensation')
plt.show()

Here is the result
If you think it looks like an impulse response, continue to the next cell
If you want to redo the phase compensation,go back to the cell "6.1 Phase compensation" 
(you will have the opportunity to modify the time origin later 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 7.1 Play with time origin 

In [9]:
time_1=time[0,:]
freq_1=freq[0,:]
print('You must now define the time origin')
print('Click once on the spectrogram at the position where you want to define the time origin')
interactive(True)
point= pts_select(abs(tfr_sig_rec[:512])**2,time_1,freq_1)
print('When you have click to define the time origin, click "disconnect mpl" and continue') 

You must now define the time origin
Click once on the spectrogram at the position where you want to define the time origin


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have click to define the time origin, click "disconnect mpl" and continue


## 7. 2 Shorten the signal, make it start at the chosen time origin, and warp it

In [10]:
# we extract the selected time origin 
pts=point.selected_points
pts_a=np.array(pts)
t_dec=pts_a[0,0]
    
## Shorten the signal, make it start at the chosen time origin, and warp it
time_t_dec=np.abs(time-t_dec)
ind0=np.where(time_t_dec==np.min(time_t_dec))
ind0=ind0[1]
ind0=ind0[0]

s_ok=sig_rec_t[ind0:]
N_ok=len(s_ok)



#Corresponding time and frequency axis
time_ok=np.arange(0,N_ok)/fs
    
#The warped signal will be s_w
s_ok_1=np.transpose(s_ok)
s_w, fs_w=warp_temp_exa(s_ok_1,fs,r,c1)
M=len(s_w)


### Original signal
N_window=31  # you need a short window to see the modes
a=np.arange(1,N_ok+1)
a=a[np.newaxis,:]
h=np.hamming(N_window)
h=h[:,np.newaxis]
tfr=tfrstft(s_ok,a,NFFT,h)
spectro=abs(tfr)**2


### Warped signal
N_window_w=301  # You need a long window to see the warped modes
wind=np.hamming(N_window_w)
wind=wind/np.linalg.norm(wind)
wind=wind[:,np.newaxis]
b=np.arange(1,M+1)
b=b[np.newaxis,:]
tfr_w=tfrstft(s_w,b,NFFT,wind)
spectro_w=abs(tfr_w)**2


# Time and frequency axis of the warped signal
time_w=np.arange(0,(M)/fs_w, 1/fs_w)
freq_w=np.arange(0,fs_w-fs_w/NFFT+fs_w/NFFT,fs_w/NFFT)


# Figure

print('The left panel shows the spectrogram of the original ')
print('signal with the chosen time origin')
print('The right panel shows the corresponding warped signal')
print('Repeat the previous cell if you want to redo the time origin selection')
print('Continue if you want to proceed with modal filtering ')
print('')

plt.figure(figsize=(10,8))
plt.subplot(121)
plt.imshow( spectro, extent=[time_ok[0,0], time_ok[0,-1], freq[0,0], freq[0,-1]] ,aspect='auto', origin='low')
plt.ylim([0, fs/2])
plt.xlim([0, 0.5])  ### Adjust this to see better
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Original signal with chosen time origin')



plt.subplot(122)
plt.imshow(spectro_w, extent=[time_w[0], time_w[-1], freq_w[0], freq_w[-1]] ,aspect='auto', origin='low')
plt.ylim([0,40]) ### Adjust this to see better
plt.xlabel('Warped time (sec)')
plt.ylabel('Corresponding warped frequency (Hz)')
plt.title('Corresponding warped signal')
plt.show()

print('Go back to the cell "7.1 Play with time origin", if you want to redo the time origin selection')
print('Continue if you want to proceed with modal filtering')

The left panel shows the spectrogram of the original 
signal with the chosen time origin
The right panel shows the corresponding warped signal
Repeat the previous cell if you want to redo the time origin selection
Continue if you want to proceed with modal filtering 



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Go back to the cell "7.1 Play with time origin", if you want to redo the time origin selection
Continue if you want to proceed with modal filtering


## 8. Filtering

#### Filter mode 1

In [11]:
## selection the spectro_w part to mask 

spectro_w_1=spectro_w[0:200,:]
time_w_1=time_w
freq_w_1=freq_w[:200]
Nmode=4
modes=np.zeros((N_ok,Nmode))
tm=np.zeros((NFFT,Nmode))


# To make it easier, filtering will be done by hand using the roipoly tool.
# See python help for more information

print('Now try to filter the 4 modes');
print('Create the four masks sequentially, starting with mode 1, then 2, etc.')
print('To do so, click twice n the spectrogram to create a line and continue to define the region you want to filter ')
print('Once you are ok with the mask shape, click "Disconect mpl" and continue')
print('(if needed, go back to c_warping_and_filtering.m for mask creation)')
print('')

# Let filter a mode 1
print('Filter mode 1')
interactive(True)
section=bbox_select(spectro_w_1)
#section=bbox_select(spectro_w_1,time_w_1,
print('Continue to filter mode 2') 

Now try to filter the 4 modes
Create the four masks sequentially, starting with mode 1, then 2, etc.
To do so, click twice n the spectrogram to create a line and continue to define the region you want to filter 
Once you are ok with the mask shape, click "Disconect mpl" and continue
(if needed, go back to c_warping_and_filtering.m for mask creation)

Filter mode 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

Continue to filter mode 2


#### Filter mode 2

In [12]:
# create the mask of the section
pts=section.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))

masque_1=np.double(mask)

# add the part masked to the total sprectogram array
masque_0=np.zeros_like(spectro_w[200:,:])
masque=np.concatenate((masque_1,masque_0),axis=0)


mode_rtf_warp=masque*tfr_w
norm=1/NFFT/np.max(wind)
mode_temp_warp=np.real(np.sum(mode_rtf_warp,axis=0))*norm*2
mode=iwarp_temp_exa(mode_temp_warp,fs_w,r,c1,fs,N_ok)
modes[:,0]=mode[:,0]

## Verification

a=hilbert(mode)
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
h=np.hamming(N_window)
h=h[:,np.newaxis]
mode_stft=tfrstft(a,b,NFFT,h)
mode_spectro=abs(mode_stft)**2
tm_1,D2=momftfr(mode_spectro,0,N_ok,time_ok)
tm[:,0]=tm_1[:,0]

## Mode 2
print('Filter mode 2')
interactive(True)
section_2=bbox_select(spectro_w_1)
print('Continue to filter mode 3') 

Filter mode 2


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

Continue to filter mode 3


#### Filter mode 3

In [13]:
# create the mask of the section
pts=section_2.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))

masque_2=np.double(mask)

# add the part masked to the total sprectogram array
masque_2=np.concatenate((masque_2,masque_0),axis=0)


mode_rtf_warp_2=masque_2*tfr_w
mode_temp_warp_2=np.real(np.sum(mode_rtf_warp_2,axis=0))*norm*2
mode_2=iwarp_temp_exa(mode_temp_warp_2,fs_w,r,c1,fs,N_ok)
modes[:,1]=mode_2[:,0]

## Verification
a_2=hilbert(mode_2)
mode_stft_2=tfrstft(a_2,b,NFFT,h)
mode_spectro_2=abs(mode_stft_2)**2
tm_2,D2_2=momftfr(mode_spectro_2,0,N_ok,time_ok)
tm[:,1]=tm_2[:,0]
## Mode 3
print('Filter mode 3')
interactive(True)
section_3=bbox_select(spectro_w_1)
print('Continue to filter mode 4') 

Filter mode 3


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

Continue to filter mode 4


#### Filter mode 4

In [14]:
# create the mask of the section
pts=section_3.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))

masque_3=np.double(mask)

# add the part masked to the total sprectogram array
masque_3=np.concatenate((masque_3,masque_0),axis=0)


mode_rtf_warp_3=masque_3*tfr_w
mode_temp_warp_3=np.real(np.sum(mode_rtf_warp_3,axis=0))*norm*2
mode_3=iwarp_temp_exa(mode_temp_warp_3,fs_w,r,c1,fs,N_ok)
modes[:,2]=mode_3[:,0]

## Verification

a_3=hilbert(mode_3)
mode_stft_3=tfrstft(a_3,b,NFFT,h)
mode_spectro_3=abs(mode_stft_3)**2
tm_3,D2_3=momftfr(mode_spectro_3,0,N_ok,time_ok)
tm[:,2]=tm_3[:,0]

## Mode 4
print('Filter mode 4')
interactive(True)
section_4=bbox_select(spectro_w_1)
print('When you have finish,click "disconnect mpl" and continue') 

Filter mode 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have finish,click "disconnect mpl" and continue


In [15]:
# create the mask of the section
pts=section_4.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))
masque_4=np.double(mask)

# add the part masked to the total sprectogram array
masque_4=np.concatenate((masque_4,masque_0),axis=0)


mode_rtf_warp_4=masque_4*tfr_w
mode_temp_warp_4=np.real(np.sum(mode_rtf_warp_4,axis=0))*norm*2
mode_4=iwarp_temp_exa(mode_temp_warp_4,fs_w,r,c1,fs,N_ok)
modes[:,3]=mode_4[:,0]

## Verification

a_4=hilbert(mode_4)
mode_stft_4=tfrstft(a_4,b,NFFT,h)
mode_spectro_4=abs(mode_stft_4)**2
tm_4,D2=momftfr(mode_spectro_4,0,N_ok,time_ok)
tm[:,3]=tm_4[:,0]

print('End of filtering')
print('Continue')

End of filtering
Continue


## 9. Verification

In [16]:
print('The red lines are the estimated dispersion curves.')
print('For real life applications, you will have to restrict them to a frequency band where they are ok')
print('Now we need to undo the phase compensation to look at the true filtered mode')


plt.figure(figsize=[10,7])
plt.imshow(spectro, extent=[time_ok[0,0], time_ok[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low')
plt.ylim([0,fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram and estimated dispersion curve')
plt.plot(tm[:,0],freq[0, :],'r')
plt.plot(tm[:,1],freq[0, :],'r')
plt.plot(tm[:,2],freq[0, :],'r')
plt.plot(tm[:,3],freq[0, :],'r')
plt.show()



print('Continue to look at your result vs the true modes')


The red lines are the estimated dispersion curves.
For real life applications, you will have to restrict them to a frequency band where they are ok
Now we need to undo the phase compensation to look at the true filtered mode


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue to look at your result vs the true modes


## 10. Let's undo phase compensation

In [17]:
### look at e_warping_and_filtering_and_phase_comp.m if you want to undo
### phase compensation for the filtered modes. Here, we will only undo it
### for the dispersion curves

data = sio.loadmat(os.getcwd()+ '/sig_pek_and_modes_for_warp.mat')
r=data['r']
vg=data['vg']
c1=data['c1']
f_vg=data['f_vg']

#### we need a few tricks to create the theoretical dispersion curves
###### first the dispersion curves of the impulse response

tm_theo_ir=r/vg-r/c1 ### range over group_speed minus correction for time origin

##### now we need to include the source law ....
f_source=IFLAW_source*fs
t_source=np.arange(0,len(IFLAW_source))/fs

f = interpolate.interp1d(f_source[0,:], t_source[0,:], bounds_error=False, fill_value=np.nan )
t_source_ok = f(f_vg[0,:]) 
t_source_ok_1=np.tile(t_source_ok,(5,1))
tm_theo_with_source=tm_theo_ir+t_source_ok_1


#### Now we modify our estimated dispersion curve to undo phase compensation
###### first we define the time-frequency law of our estimated source

f_source_est=IFLAW_est*fs
t_source_est=np.arange(0,len(IFLAW_est))/fs

##### we need it on the same frequency axis than the dispersion curves

f = interpolate.interp1d(f_source_est[:,0], t_source_est[0,:], bounds_error=False, fill_value=np.nan )
t_source_est_ok = f(freq[0,:]) 
t_source_est_ok=t_source_est_ok[:,np.newaxis]
t_source_est_ok_1=np.tile(t_source_ok,(4,1))
tm_est_with_source=tm+t_source_est_ok+t_dec


print('This is the spectrogram of the original signal,')
print('the true dispersion curves (black) and your estimation (red).')
print('Remember that the estimated dispersion curve are ')
print('relevant only in the frequency band where they match')
print('the estimated spectrogram.')
print('If you do not like the result, try to redo phase compensation and/or change time origin')



plt.figure()
plt.imshow(spectro_init, extent=[time_init[0,0], time_init[0,-1], freq_init[0,0], freq_init[0,-1]],aspect='auto', origin='low' )
plt.xlim([0,1.2])
plt.ylim([0,fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Original signal and estimated dispersion curve')

plt.plot(tm_theo_with_source[0,:], f_vg[0,:], 'black')
plt.plot(tm_theo_with_source[1,:], f_vg[0,:], 'black')
plt.plot(tm_theo_with_source[2,:], f_vg[0,:], 'black')
plt.plot(tm_theo_with_source[3,:], f_vg[0,:], 'black')
plt.plot(tm_est_with_source[:,0],freq[0,:], linewidth=2, color='r')
plt.plot(tm_est_with_source[:,1],freq[0,:], linewidth=2, color='r')
plt.plot(tm_est_with_source[:,2],freq[0,:], linewidth=2, color='r')
plt.plot(tm_est_with_source[:,3],freq[0,:], linewidth=2, color='r')


plt.show()

print(' ')
print('END')

This is the spectrogram of the original signal,
the true dispersion curves (black) and your estimation (red).
Remember that the estimated dispersion curve are 
relevant only in the frequency band where they match
the estimated spectrogram.
If you do not like the result, try to redo phase compensation and/or change time origin


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 
END
